<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/LangStudio/SMA_Model_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas_ta
!pip install pandas==2.2.2 numpy==1.26.0 --force-reinstall
# After running this cell, please restart the Colab runtime (Runtime -> Restart runtime) to apply the changes.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.3
    Uninstalling tzdata-2025.3:
      Successfully uninstalled tzdata-2025.3
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6

In [1]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

def run_backtest(symbol, sma_long=200, sma_short=5):
    # 1. Download Data
    df = yf.download(symbol, period="5y", interval="1d")

    # 2. Calculate Indicators (Vectorized)
    df['SMA_L'] = ta.sma(df['Close'], length=sma_long)
    df['SMA_S'] = ta.sma(df['Close'], length=sma_short)
    bb = ta.bbands(df['Close'], length=20, std=2)

    # Dynamically find columns as we did in the agent
    bbl_col = [c for c in bb.columns if c.startswith('BBL')][0]
    bbm_col = [c for c in bb.columns if c.startswith('BBM')][0]

    df['Low_B'] = bb[bbl_col]
    df['Mid_B'] = bb[bbm_col]
    df['Value_Zone_Ceiling'] = (df['Mid_B'] + df['Low_B']) / 2

    # 3. Define Conditions (Boolean Masks)
    condition_1 = df['Close'] > df['SMA_L']                 # Long term uptrend
    condition_2 = (df['Close'] >= df['Low_B']) & (df['Close'] <= df['Value_Zone_Ceiling']) # Value Zone
    condition_3 = df['Close'] > df['SMA_S']                 # Short term momentum

    # 4. Generate Signals
    df['Signal'] = (condition_1 & condition_2 & condition_3).astype(int)

    # 5. Simple Returns Calculation
    # Assuming we buy at close when signal is true and hold for 5 days
    df['Next_Returns'] = df['Close'].pct_change(5).shift(-5)

    # Performance of the strategy vs Buy & Hold
    strategy_returns = df[df['Signal'] == 1]['Next_Returns'].mean()

    return strategy_returns

# Example Run
result = run_backtest("TSLA", 200, 5)
print(f"Average 5-day return when signals trigger: {result:.2%}")

/tmp/ipython-input-1714980573.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="5y", interval="1d")
[*********************100%***********************]  1 of 1 completed


AttributeError: 'NoneType' object has no attribute 'columns'